# An sample to finetune vicuna on SageMaker

In [ ]:
## Update sagemaker python sdk version
!pip install -U sagemaker

In [ ]:
import sagemaker
import boto3
from sagemaker import get_execution_role

sess = sagemaker.Session()
role = get_execution_role()
sagemaker_default_bucket = sess.default_bucket()

account = sess.boto_session.client("sts").get_caller_identity()["Account"]
region = sess.boto_session.region_name

In [ ]:
## download training script from github
!git clone https://github.com/lm-sys/FastChat.git

## Download pretrained model from HuggingFace Hub

To avoid download model from Huggingface hub failure, we download first and push those model files to S3 bucket first.

In [ ]:
!pip install huggingface_hub

In [ ]:
from huggingface_hub import snapshot_download
from pathlib import Path

local_cache_path = Path("./model")
local_cache_path.mkdir(exist_ok=True)

model_name = "pinkmanlove/llama-7b-hf"#decapoda-research/llama-13b-hf

# Only download pytorch checkpoint files
allow_patterns = ["*.json", "*.pt", "*.bin", "*.model"]

model_download_path = snapshot_download(
    repo_id=model_name,
    cache_dir=local_cache_path,
    allow_patterns=allow_patterns,
)

**Upload model files to S3**

In [ ]:
# Get the model files path
import os
from glob import glob

local_model_path = None

paths = os.walk(r'./model')
for root, dirs, files in paths:
    for file in files:
        if file == 'config.json':
            print(os.path.join(root,file))
            local_model_path = str(os.path.join(root,file))[0:-11]
            print(local_model_path)
if local_model_path == None:
    print("Model download may failed, please check prior step!")

In [ ]:
%%script env sagemaker_default_bucket=$sagemaker_default_bucket local_model_path=$local_model_path bash

chmod +x ./s5cmd
./s5cmd sync ${local_model_path} s3://${sagemaker_default_bucket}/llama/pretrain/pinkmanlove/llama-7b-hf/ 

rm -rf model

## Prepare docker image

In [ ]:
%%writefile Dockerfile
## You should change below region code to the region you used, here sample is use us-west-2
From 763104351884.dkr.ecr.us-west-2.amazonaws.com/huggingface-pytorch-training:1.13.1-transformers4.26.0-gpu-py39-cu117-ubuntu20.04 

ENV LANG=C.UTF-8
ENV PYTHONUNBUFFERED=TRUE
ENV PYTHONDONTWRITEBYTECODE=TRUE

# RUN python3 -m pip install git+https://github.com/huggingface/transformers.git@97a3d16a6941294d7d76d24f36f26617d224278e

RUN pip3 uninstall -y deepspeed && pip3 install deepspeed

## Make all local GPUs visible
ENV NVIDIA_VISIBLE_DEVICES="all"

In [ ]:
## You should change below region code to the region you used, here sample is use us-west-2
!aws ecr get-login-password --region us-west-2 | docker login --username AWS --password-stdin 763104351884.dkr.ecr.us-west-2.amazonaws.com

**Build image and push to ECR.**

In [ ]:
## define repo name, should contain *sagemaker* in the name
repo_name = "sagemaker-vicuna-demo"

In [ ]:
%%script env repo_name=$repo_name bash

#!/usr/bin/env bash

# This script shows how to build the Docker image and push it to ECR to be ready for use
# by SageMaker.

# The argument to this script is the image name. This will be used as the image on the local
# machine and combined with the account and region to form the repository name for ECR.
# The name of our algorithm
algorithm_name=${repo_name}

account=$(aws sts get-caller-identity --query Account --output text)

# Get the region defined in the current configuration (default to us-west-2 if none defined)
region=$(aws configure get region)
region=${region:-us-west-2}

fullname="${account}.dkr.ecr.${region}.amazonaws.com/${algorithm_name}:latest"

# If the repository doesn't exist in ECR, create it.
aws ecr describe-repositories --repository-names "${algorithm_name}" > /dev/null 2>&1

if [ $? -ne 0 ]
then
    aws ecr create-repository --repository-name "${algorithm_name}" > /dev/null
fi

# Get the login command from ECR and execute it directly
aws ecr get-login-password --region ${region}|docker login --username AWS --password-stdin ${fullname}

# Build the docker image locally with the image name and then push it to ECR
# with the full name.

docker build -t ${algorithm_name} .
docker tag ${algorithm_name} ${fullname}

docker push ${fullname}

### Generate the deepspeed config

In [ ]:
%%writefile ds.json
{
  "fp16": {
    "enabled": true,
    "auto_cast": false,
    "loss_scale": 0,
    "initial_scale_power": 16,
    "loss_scale_window": 1000,
    "hysteresis": 2,
    "min_loss_scale": 1
  },
  "optimizer": {
    "type": "AdamW",
    "params": {
      "lr": "auto",
      "betas": "auto",
      "eps": "auto",
      "weight_decay": "auto"
    }
  },
  "scheduler": {
    "type": "WarmupLR",
    "params": {
      "warmup_min_lr": "auto",
      "warmup_max_lr": "auto",
      "warmup_num_steps": "auto"
    }
  },
  "zero_optimization": {
    "stage": 3,
    "overlap_comm": true,
    "contiguous_gradients": true,
    "sub_group_size": 1e9,
    "reduce_bucket_size": "auto",
    "stage3_prefetch_bucket_size": "auto",
    "stage3_param_persistence_threshold": "auto",
    "stage3_max_live_parameters": 1e9,
    "stage3_max_reuse_distance": 1e9,
    "stage3_gather_16bit_weights_on_model_save": true
  },
  "gradient_accumulation_steps": "auto",
  "gradient_clipping": "auto",
  "steps_per_print": 2000,
  "train_batch_size": "auto",
  "train_micro_batch_size_per_gpu": "auto",
  "wall_clock_breakdown": false
}

**Generate training entrypoint script.**

**Note: DO NOT CHANGE BELOW VAlUE OF "output_dir" and "cache_dir", keep it "/tmp/llama_out" and "/tmp".**

Below is just a testing to fine-tune on a sample dataset (just 8 samples), you could change ```data_path``` to your dataset for furthur fine tune.

For the dataset download, you could follow the way how to download pretrain model:
```
./s5cmd sync s3://$MODEL_S3_BUCKET/llama/pretrain/7B/* /tmp/llama_pretrain/
```

It is recommend to use the folder ```/tmp/dataset/```.

In [ ]:
# export PYTHONPATH=./FastChat:$PYTHONPATH
# echo $PYTHONPATH
# cd FastChat && pip install -e . && cd ..

In [ ]:
# %%writefile train.sh
# #!/bin/bash

# chmod +x ./s5cmd
# ./s5cmd sync s3://$MODEL_S3_BUCKET/llama/pretrain/pinkmanlove/llama-7b-hf/* /tmp/llama_pretrain/

# cd FastChat && pip install -e . && cd ..

# torchrun --nproc_per_node=8 --master_port=20001 FastChat/fastchat/train/train_mem.py \
#     --model_name_or_path /tmp/llama_pretrain/  \
#     --data_path sharegpt_test.json \
#     --bf16 True \
#     --output_dir /tmp/llama_out \
#     --num_train_epochs 3 \
#     --per_device_train_batch_size 2 \
#     --per_device_eval_batch_size 2 \
#     --gradient_accumulation_steps 16 \
#     --evaluation_strategy "no" \
#     --save_strategy "steps" \
#     --save_steps 1200 \
#     --save_total_limit 10 \
#     --learning_rate 2e-5 \
#     --weight_decay 0. \
#     --warmup_ratio 0.03 \
#     --lr_scheduler_type "cosine" \
#     --logging_steps 1 \
#     --fsdp "full_shard auto_wrap" \
#     --fsdp_transformer_layer_cls_to_wrap 'LlamaDecoderLayer' \
#     --tf32 True \
#     --model_max_length 2048 \
#     --gradient_checkpointing True \
#     --lazy_preprocess True \
#     --report_to "none"

# if [ $? -eq 1 ]; then
#     echo "Training script error, please check CloudWatch logs"
#     exit 1
# fi

# ./s5cmd sync /tmp/llama_out s3://$MODEL_S3_BUCKET/llama/output/$(date +%Y-%m-%d-%H-%M-%S)/

We will use deepspeed launch instead of torch run.

(When use torch run, there will report tensor size not match)

## Notice

We modified some parts of ```FastChat/fastchat/train/train.py```, such as how to save model.

Here we use sample dataset - sharegpt_test.json for testing.

In [ ]:
%%writefile ds-train.sh
#!/bin/bash

chmod +x ./s5cmd
./s5cmd sync s3://$MODEL_S3_BUCKET/llama/pretrain/pinkmanlove/llama-7b-hf/* /tmp/llama_pretrain/

cd FastChat && pip install -e . && cd ..


deepspeed --num_gpus=8 FastChat/fastchat/train/train_mem.py \
    --deepspeed ds.json \
    --model_name_or_path "/tmp/llama_pretrain/" \
    --data_path sharegpt_test.json \
    --output_dir "/tmp/llama_out" \
    --num_train_epochs 1 \
    --per_device_train_batch_size 2 \
    --per_device_eval_batch_size  2 \
    --gradient_accumulation_steps 8 \
    --evaluation_strategy "no" \
    --save_strategy "no" \
    --save_steps 2000 \
    --save_total_limit 1 \
    --learning_rate 2e-5 \
    --weight_decay 0. \
    --warmup_ratio 0.03 \
    --lr_scheduler_type "cosine" \
    --logging_steps 1 \
    --cache_dir '/tmp' \
    --model_max_length 2048 \
    --gradient_checkpointing True \
    --lazy_preprocess True \
    --fp16 True \
    --tf32 True \
    --report_to "none"

if [ $? -eq 1 ]; then
    echo "Training script error, please check CloudWatch logs"
    exit 1
fi

./s5cmd sync /tmp/llama_out s3://$MODEL_S3_BUCKET/llama/output/$(date +%Y-%m-%d-%H-%M-%S)/

In [ ]:
## The image uri which is build and pushed above
image_uri = "{}.dkr.ecr.{}.amazonaws.com/{}:latest".format(account, region, repo_name)
image_uri

In [ ]:
## set train_data_path to your training dataset path in s3
train_data_path = f's3://{sagemaker_default_bucket}/llama/train_data/'

inputs = {'train': train_data_path}

In [ ]:
import time
from sagemaker.estimator import Estimator

environment = {
              'MODEL_S3_BUCKET': sagemaker_default_bucket # The bucket to store pretrained model and fine-tune model
}

base_job_name = 'vicuna-demo'         

instance_type = 'ml.p4d.24xlarge'

estimator = Estimator(role=role,
                      entry_point='ds-train.sh',
                      source_dir='./',
                      base_job_name=base_job_name,
                      instance_count=1,
                      instance_type=instance_type,
                      image_uri=image_uri,
                      environment=environment,
                      disable_profiler=True,
                      debugger_hook_config=False)

estimator.fit()
# estimator.fit(inputs)

You could find the model path in S3 from above logs.